# Imports

In [ ]:
from pathlib import Path
import pandas as pd, numpy as np
import plotly.express as px
from datetime import datetime

In [ ]:
csvs = list(Path("/Users/alexpayne/lilac-mount-point/asap-datasets/retro_docking/sars_fragalysis_retrospective/20230411").glob("run_docking_oe.*-results.csv"))

In [ ]:
len(csvs)

## loadall csvs

In [ ]:
print("start", datetime.now())
df = pd.concat(map(pd.read_csv, csvs), ignore_index= True)
print("end", datetime.now())

In [ ]:
from asapdiscovery.docking.analysis import DockingResults

In [ ]:
dr = DockingResults(df=df, column_names="None")

In [ ]:
dr.df.columns

In [ ]:
dr.df.columns = ["Compound_ID", 
                 "Structure_Source", 
                 "Docked_File", 
                 "Pose_ID", 
                 "RMSD", 
                 "POSIT", 
                 "POSIT_Method", 
                 "Chemgauss4", 
                 "Clash", 
                 "SMILES", 
                 "GAT_Score"]

In [ ]:
dr.df["POSIT_R"] = 1-dr.df["POSIT"]
dr.df["Complex_ID"] = dr.df.Compound_ID.apply(str) + "_" + dr.df.Structure_Source

In [ ]:
structure_fns = list(Path("/Users/alexpayne/lilac-mount-point/asap-datasets/full_frag_prepped_mpro_20230122").glob("*/*prepped_receptor_0.pdb"))

In [ ]:
structure_fns

In [ ]:
lilac_path = Path("/lila/data/chodera")

In [ ]:
final = lilac_path.joinpath(*structure_fns[0].parts[4:])

In [ ]:
final

In [ ]:
paths = [lilac_path.joinpath(*structure_fn.parts[4:]) for structure_fn in structure_fns]

In [ ]:
structure_paths = []
for structure in dr.df["Structure_Source"].unique():
    for path in paths:
        if structure in str(path):
            structure_paths.append(path)

In [ ]:
dr.df.Structure_Source

In [ ]:
structure_source_df = pd.DataFrame({"Structure_Source": dr.df["Structure_Source"].unique(), "Structure_Path": structure_paths})

In [ ]:
dr.df = pd.merge(dr.df, structure_source_df, on="Structure_Source")

In [ ]:
dr.df

# Write results

In [ ]:
output_dir = csvs[0].parent

In [ ]:
dr.df.to_csv(output_dir / "all_results_cleaned.csv")

# Load Results

In [ ]:
main_path = Path("/Users/alexpayne/lilac-mount-point/asap-datasets/retro_docking/sars_fragalysis_retrospective/20230411")

In [ ]:
csv = main_path/"all_results_cleaned.csv"

In [ ]:
csv.exists()

## load as pandas df

In [ ]:
df = pd.read_csv(csv, index_col=0)

# Add compound info

In [ ]:
import yaml

In [ ]:
with open("../../../../metadata/cmpd_to_frag.yaml") as f:
    cmpd_to_frag_dict = yaml.safe_load(f)

In [ ]:
dataset_df = pd.DataFrame({"Compound_ID": cmpd_to_frag_dict.keys(), 
                           "Compound_Source": [source for source in cmpd_to_frag_dict.values()]})

In [ ]:
dataset_df

In [ ]:
df = pd.merge(df, dataset_df, on="Compound_ID")

In [ ]:
df.to_csv(csv)